In [8]:
import os
import pandas as pd

In [9]:
FILE = os.path.join(os.getcwd(), 'checked_list.csv')
df = pd.read_csv(FILE, encoding='utf-8-sig')
df['check'] = df[['check']].astype('Int64')

In [10]:
df['d'] = [0 if d is pd.NA else d for d in df['check'] - df['总数量']] 
df.dtypes

库位       object
料品       object
品名       object
批号       object
可用数量      int64
冻结数量      int64
总数量       int64
check    object
d         int64
dtype: object

In [11]:
for sku, df_sku in df[df['d'] != 0].groupby('料品'):
    display(df_sku)
    

,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
285,4AH5,100101-25,太粮王泰国茉莉香米25,200817005,60,0,60,48,-12
411,4AP6,100101-25,太粮王泰国茉莉香米25,200817005,0,0,0,12,12


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
0,3A00,100101-35,太粮王泰国茉莉香米5*4,200101001,4,0,4,6,2
102,4A00,100101-35,太粮王泰国茉莉香米5*4,191119004,5,0,5,1,-4
103,4A00,100101-35,太粮王泰国茉莉香米5*4,191119006,29,0,29,31,2


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
17,3A01,100101-40,太粮王泰国茉莉香米10*3,191119003,11,0,11,1,-10
18,3A01,100101-40,太粮王泰国茉莉香米10*3,200101001,20,0,20,30,10


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
32,3A02,100501-35,柬埔寨茉莉香米5*5,200814006,170,0,170,360,190
38,3A04,100501-35,柬埔寨茉莉香米5*5,200814006,648,0,648,616,-32
60,3A08,100501-35,柬埔寨茉莉香米5*5,200814006,648,0,648,490,-158


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
90,3AP1,100502-35,（橙色）柬埔寨茉莉香米5*5,200617002,39,0,39,0,-39
91,3AP1,100502-35,（橙色）柬埔寨茉莉香米5*5,200620003,30,0,30,69,39


...

,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
15,3A00,150103-35,鸣象富桂银粘米5*5,200812007,44,0,44,29,-15
31,3A01,150103-35,鸣象富桂银粘米5*5,200818010,120,0,120,108,-12
127,4A00,150103-35,鸣象富桂银粘米5*5,200812007,27,0,27,54,27


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
68,3A08,150107-35,鸣象（丝苗）米5*5,200810014,4,0,4,8,4
89,3A09,150107-35,鸣象（丝苗）米5*5,200816001,195,0,195,191,-4


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
135,4A01,160101-10,帝皇鹰米满香米10,200810001,936,0,936,880,-56
191,4A07,160101-10,帝皇鹰米满香米10,200810001,153,0,153,33,-120
333,4B01,160101-10,帝皇鹰米满香米10,200802003,339,0,339,515,176


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
137,4A01,160102-25,帝皇鹰喜之粮米25,200715051,13,0,13,6,-7
163,4A04,160102-25,帝皇鹰喜之粮米25,200716036,553,0,553,483,-70
164,4A04,160102-25,帝皇鹰喜之粮米25,200802003,50,0,50,28,-22
335,4B01,160102-25,帝皇鹰喜之粮米25,200815019,80,0,80,78,-2
423,4A01,160102-25,帝皇鹰喜之粮米25,200716036,0,0,0,91,91


,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d
128,4A00,160204-35,喜之粮湖蟹卧田米5*5,200805013,259,0,259,184,-75
424,3A06,160204-35,喜之粮湖蟹卧田米5*5,200805013,0,0,0,66,66
